<a href="https://colab.research.google.com/github/hananather/AutoText/blob/main/GPT_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-08-12 14:37:04--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2024-08-12 14:37:04 (18.0 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [4]:
len(text)

1115394

In [5]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



Our objective is to tokenize each character to an integer.

In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(chars)
stoi = {s: i for i,s in enumerate(chars)}
itos = {i:s for i,s in enumerate(chars)}

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [9]:
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [10]:
print(encode("Hanan"))
print(decode(encode("Hanan")))

[20, 39, 52, 39, 52]
Hanan


In [13]:
# Encode the entire dataset and store it into torch.Tensor
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape,  data.dtype)

torch.Size([1115394]) torch.int64


In [14]:
# split the data into train and validation split
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [15]:
block_size = 8
train_data[:block_size +1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [16]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"input is: {context} targer is: {target}")

input is: tensor([18]) targer is: 47
input is: tensor([18, 47]) targer is: 56
input is: tensor([18, 47, 56]) targer is: 57
input is: tensor([18, 47, 56, 57]) targer is: 58
input is: tensor([18, 47, 56, 57, 58]) targer is: 1
input is: tensor([18, 47, 56, 57, 58,  1]) targer is: 15
input is: tensor([18, 47, 56, 57, 58,  1, 15]) targer is: 47
input is: tensor([18, 47, 56, 57, 58,  1, 15, 47]) targer is: 58


In [59]:
torch.manual_seed(1337)
batch_size = 4 # number of independent sequences
block_size = 8

def get_batch(split):
  # generate a small batch of data
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i: i +block_size] for i in ix])
  y = torch.stack([data[i+1: i+block_size +1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)
print('----------')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----------
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53, 5

In [42]:
import torch
import  torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets):

    logits = self.token_embedding_table(idx) # (B,T,C)

    return logits

m = BigramLanguageModel(vocab_size)
out = m(xb,yb)
print(out)
print(out.shape)

tensor([[[-1.5101, -0.0948,  1.0927,  ..., -0.6126, -0.6597,  0.7624],
         [ 0.3323, -0.0872, -0.7470,  ..., -0.6716, -0.9572, -0.9594],
         [ 0.2475, -0.6349, -1.2909,  ...,  1.3064, -0.2256, -1.8305],
         ...,
         [ 0.5978, -0.0514, -0.0646,  ..., -1.4649, -2.0555,  1.8275],
         [ 1.0901,  0.2170, -2.9996,  ..., -0.5472, -0.8017,  0.7761],
         [ 0.3323, -0.0872, -0.7470,  ..., -0.6716, -0.9572, -0.9594]],

        [[ 1.0541,  1.5018, -0.5266,  ...,  1.8574,  1.5249,  1.3035],
         [-0.1324, -0.5489,  0.1024,  ..., -0.8599, -1.6050, -0.6985],
         [-0.6722,  0.2322, -0.1632,  ...,  0.1390,  0.7560,  0.4296],
         ...,
         [ 1.0901,  0.2170, -2.9996,  ..., -0.5472, -0.8017,  0.7761],
         [ 1.1513,  1.0539,  3.4105,  ..., -0.5686,  0.9079, -0.1701],
         [ 0.2475, -0.6349, -1.2909,  ...,  1.3064, -0.2256, -1.8305]],

        [[-0.2103,  0.4481,  1.2381,  ...,  1.3597, -0.0821,  0.3909],
         [ 0.2475, -0.6349, -1.2909,  ...,  1

**Embedding**
A simple lookup table that stores embeddings of fixed dictionary and size.

This module is often used to store word embeddings and retrieve them using indicies. The input to the module is a list of indicies, and the output is the corresponding word embeddings.

**Cross Entropy Loss**
Computes the cross entropy loss between input logits and target.
- the input is expected to contain the unnormalized logits for each class (which we don't need to be positie for sum to 1, in general)
- input has to be a tensor of of size (minibatch, C)

In [51]:
import torch
import  torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets = None):
    logits = self.token_embedding_table(idx) # (B,T,C)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indicies in the current context
    for _ in range(max_new_tokens):
      # get the pridictions
      logits, loss = self(idx)
      # focus only on the last time step
      logits = logits[:,-1,:] # becomes (B,C)
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=1) # (B, C)
      # sample from distribution
      idx_next = torch.multinomial(probs, num_samples = 1) #( B, 1)
      # append sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb,yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [55]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype= torch.long), max_new_tokens=100)[0].tolist()))


Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [56]:
# create a Pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [62]:
batch_size  = 32
for steps in range(10000):

  # sample a batch of data
  xb, yb = get_batch('train')

  # evaluate the loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none = True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.3997652530670166


In [63]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype= torch.long), max_new_tokens=100)[0].tolist()))


HUCI 'lilvit 't mig ater a lethye wde wasthod f mb.
WA:

ad shalYouers! brema'suD bems as II wit sir


In [64]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype= torch.long), max_new_tokens=300)[0].tolist()))





INurugy I's heleveind.
Walo hilleicu to athief thed blesst' pr,
CHENo O: l,
LOn taior dve
lit,
Im mothy;
ARLALI d att unt'darisreathousefthice coexace ton hoth t ffe oithi'dito I I Ino truret
anathopusuthig ofarid ke s wherelak, veirrfaichired t ctor s ke,
DWh gund the;
xceat y, at o CO:-n thuf t
